
# LINEAR REGRESSION WITH PYTHON
## Includes Stepwise and cross validation

## Steps:
#### 1. Initializing and data audit
#### 2. Select relevant variables: stepwise
#### 3. Data exploration
#### 4. SKLEARN execution
    A. Linear Regression with data splitting
    B. Model Evaluation
    C. Predictions and Error Measures
#### 5. STATSMODEL
    A. Linear Regression withouth data splitting
    B. Main results
    C. Predictions and Error Measures
    D. Stores predictions and errors
    E. Execute model on reserved dataset and stores predictions and errors



## Import libraries


In [0]:
%pylab inline

In [0]:
import dataiku                               # Access to Dataiku datasets
import dataikuapi                            # API to create new datasets
import pandas as pd, numpy as np             # Data manipulation 
from dataiku import pandasutils as pdu
from matplotlib import pyplot as plt         # Graphing
import seaborn as sns                        # Graphing
import statsmodels.api as sm                    # Statistical analysis
#sns.set(style="white")                       # Tuning the style of charts
import warnings                              # Disable some warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from scipy import stats                      # Stats

## Initializes variables

In [0]:
dataset_name = "wage_model"
dataset_name_reserved = "wage_reserved"
dependent = "WAGE"


## Check out data


In [0]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset(dataset_name)
mydataset_df = mydataset.get_dataframe()

In [0]:
mydataset_df.head() 

In [0]:
pd.set_option("display.precision", 2)
mydataset_df.describe()

In [0]:
# Get some simple descriptive statistics
pdu.audit(mydataset_df)

## Stepwise execution

In [0]:
# Stepwise function
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.1, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(X[included])).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [0]:
# Removes variables with missing values, stepwise does not work with them
selected_fields=mydataset_df.drop(labels=[dependent], axis=1)
for field in selected_fields:
    if selected_fields[field].isnull().any():
        selected_fields=selected_fields.drop(labels=[field], axis=1)

selected_fields

In [0]:
# Sets y
y = mydataset_df[dependent]
result = stepwise_selection(selected_fields, y)

In [0]:
print('resulting features:')
print(result)

## Select explanatory variables

In [0]:
# Adds selected features from stepwise
X=selected_fields
for item in selected_fields.columns:
    if item not in result:
        X=X.drop(labels=[item],axis=1) #removes the non relevant variables

# Exploratory Data Analysis


In [0]:
#Scatterplots of explanatory variables
sns.pairplot(X)

In [0]:
#Histogram of dependent variable
y = mydataset_df[dependent] # dependent variable
sns.distplot(y).set(title='Histogram of dependent variable')

In [0]:
#Heatmap to show correlation between explanatory variables
sns.set(font_scale=1.1)
fig, ax = plt.subplots(figsize=(10,10))         # Sample figsize in inches
sns.heatmap(X.corr(), annot=True, fmt=".2f", linewidths=1, ax=ax)

# Linear Regression Model with SKLEARN

## Train Test Split


In [0]:
# cross validation
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print ("Sample size train dataset: ", shape(X_train)[0])
print ("Sample size test dataset: ", shape(X_test)[0])

## Creating and Training the Model

In [0]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression(fit_intercept=True)
model=lm.fit(X_train, y_train)

## Model Evaluation


In [0]:
# print R squared train dataset
print("R2: ", model.score(X,y))

In [0]:
# print coefficients
coeff_df = pd.DataFrame(model.coef_,X.columns,columns=['Coefficient'])
print("Intercept: ", lm.intercept_)
coeff_df

## Predictions from our Model


In [0]:
# entire dataset
predictions = lm.predict(X)
plt.title("Real vs. Fitted (entire dataset)")
plt.scatter(y,predictions)

In [0]:
# training dataset
predictions_train = lm.predict(X_train)
plt.title("Real vs. Fitted (training dataset)")
plt.scatter(y_train,predictions_train, color="green")

In [0]:
# test dataset
predictions_test = lm.predict(X_test)
plt.title("Real vs. Fitted (test dataset)")
plt.scatter(y_test,predictions_test, color="orange")

In [0]:
# combined on same chart 
plt.title("Real vs. Fitted (train_green vs. test_yellow datasets)")
predictions = lm.predict(X_train)
plt.scatter(y_train,predictions, color="green")
predictions = lm.predict(X_test)
plt.scatter(y_test,predictions, color="orange")

## Residual Histograms

In [0]:
# entire dataset
#sns.distplot((y-predictions),bins=50)

In [0]:
warnings.filterwarnings("ignore")

# training dataset
sns.distplot((y_train-predictions_train),bins=50, color="green").set(title='Histogram of dependent - train dataset')

In [0]:
# test dataset
sns.distplot((y_test-predictions_test),bins=50, color="orange").set(title='Histogram of dependent - test dataset')

## Regression Evaluation Metrics


Here are three common evaluation metrics for regression problems:

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

**Mean Squared Error** (MSE) is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

**Root Mean Squared Error** (RMSE) is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

**Mean Absolute Percentage Error** (MAPE) is the mean of the absolute percent error:

$${\frac 1n\sum_{i=1}^n}{|y_i-\hat{y}_i)|\over y_i}$$

Comparing these metrics:

- **MAE** is the easiest to understand, because it's the average error.
- **MSE** is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.
- **RMSE** is even more popular than MSE, because RMSE is interpretable in the "y" units.
- **MAPE** is the best metric, because allows comparison between different models.

All of these are **loss functions**, because we want to minimize them.


In [0]:
from sklearn import metrics

In [0]:
# entire dataset
#print('MAE:', metrics.mean_absolute_error(y, predictions))
#print('MSE:', metrics.mean_squared_error(y, predictions))
#print('RMSE:', np.sqrt(metrics.mean_squared_error(y, predictions)))
#print('MAPE:', np.mean(100*abs(y-predictions)/y))

In [0]:
# error comparison between the two datasets
print ("Errors:\n")
print('MAE (train):', metrics.mean_absolute_error(y_train, predictions_train))
print('MAE (test):', metrics.mean_absolute_error(y_test, predictions_test))

print('MSE (train):', metrics.mean_squared_error(y_train, predictions_train))
print('MSE (test):', metrics.mean_squared_error(y_test, predictions_test))

print('RMSE (train):', np.sqrt(metrics.mean_squared_error(y_train, predictions_train)))
print('RMSE (test):', np.sqrt(metrics.mean_squared_error(y_test, predictions_test)))

print('MAPE (train):', np.mean(100*abs(y_train-predictions_train)/y_train))
print('MAPE (test):', np.mean(100*abs(y_test-predictions_test)/y_test))


# ADITIONAL INFO USING STATSMODELS LIBRARY
### (It does not include cross validation)

In [0]:
# model with an additional library (statmodels)
x = sm.add_constant(X) 

In [0]:
result = sm.OLS(y, x).fit()
print (result.summary())

In [0]:
#move dependent variable to the end to show it along with predictions and errors
mydataset_df = mydataset_df[[col for col in mydataset_df.columns if col !=dependent] + [dependent]]

In [0]:
#create predictions and convert them into a dataframe 
predictions = result.predict()
mydataset_df[dependent + "_predicted"] = pd.DataFrame(predictions, columns = [dependent + '_predicted'])

In [0]:
# calculates error, %error and MAPE
print ("Errors:\n")
mydataset_df["error"] = mydataset_df[dependent] - mydataset_df[dependent + "_predicted"]
mydataset_df["%abs error"] = abs(100*mydataset_df["error"]) /  mydataset_df[dependent]
print('MAE:', abs(mydataset_df["error"]).mean())
MSE = np.square(mydataset_df["error"]).mean()
print('MSE:', MSE) 
print('RMSE:', np.sqrt(MSE))
print ("MAPE: ", mydataset_df["%abs error"].mean())

mydataset_df

## Stores predictions and errors in new dataset

In [0]:
# Workaraound in dataiku to store results into a new dataset (not available with free license)
def create_dataset(dataset_name, df):
    client = dataiku.api_client()
    project = client.get_project(dataiku.default_project_key())
    project_variables = dataiku.get_custom_variables()
    csv_dataset_name = dataset_name + "_results"

    # Create dataset if it doesn't already exist
    try:
        # If dataset exists, clear it
        csv_dataset = project.get_dataset(csv_dataset_name) # doesn't generate error if dataset doesn't exist
        csv_dataset.clear()
    except:
        # Create dataset (assuming exception was that dataset does not exist)
        params = {'connection': 'filesystem_folders', 'path': project_variables['projectKey']  + '/' + csv_dataset_name}
        format_params = {'separator': '\t', 'style': 'unix', 'compress': ''}

        csv_dataset = project.create_dataset(csv_dataset_name, type='Filesystem', params=params, formatType='csv', formatParams=format_params)

        # Set dataset to managed
        ds_def = csv_dataset.get_definition()
        ds_def['managed'] = True
        csv_dataset.set_definition(ds_def)

    # Save results into new dataset
    output = dataiku.Dataset(csv_dataset_name)
    output.write_with_schema(df)    

In [0]:
# Creates model dataset with results
create_dataset(dataset_name, mydataset_df)

## Calculate predictions and errors from the reserved dataset

In [0]:
mydataset2 = dataiku.Dataset(dataset_name_reserved)
mydataset_df2 = mydataset2.get_dataframe()
mydataset_df2 = mydataset_df2[[col for col in mydataset_df2.columns if col !=dependent] + [dependent]]

In [0]:
# Create predictions
i=0
formula=result.params[0]
for param in result.params[1:]:
    formula += param*mydataset_df2[X.columns[i]]
    i+=1

# Store predictions end errors   
mydataset_df2[dependent + "_predicted"] = formula
mydataset_df2["error"] = mydataset_df2[dependent] - mydataset_df2[dependent + "_predicted"]
mydataset_df2["%abs error"] = abs(100*mydataset_df2["error"]) /  mydataset_df[dependent]

# Display errors    
print ("Errors from the reserved dataset:\n")
print('MAE:', abs(mydataset_df2["error"]).mean())
MSE = np.square(mydataset_df2["error"]).mean()
print('MSE:', MSE) 
print('RMSE:', np.sqrt(MSE))
print ("MAPE: ", mydataset_df2["%abs error"].mean())

In [0]:
# Creates reserved dataset with results
create_dataset(dataset_name_reserved, mydataset_df2)